In [ ]:
import sqlalchemy.engine.result
import os
import re
import glob
import shutil
import random
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime

import wandb
from wandb.integration.ultralytics import add_wandb_callback

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

from huggingface_hub import login as hf_login


load_dotenv()


ROOT_DIR = Path("../").resolve()
DATA_DIR = ROOT_DIR / "data" / "human_nonhuman"
LOG_DIR = ROOT_DIR / "logs"

WANDB_PROJECT = "human-non-human-classification"
WANDB_JOB_TYPE = "training"
WANDB_TOKEN = os.getenv("WANDB_TOKEN")


wandb.login(key=WANDB_TOKEN)

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/octoopt/.netrc
wandb: Currently logged in as: octoopt to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
!yolo settings wandb=True

✅ Updated 'wandb=True'
JSONDict("/home/octoopt/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/octoopt/workspace/projects/learn-from-basics/the-notes/cv/tracking/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "4a28363b300d4f86a8ec4580dd2079712156257fd00e1b95f3683b059b8ffdd8",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [3]:
def random_exp_name(prefix="human_nonhuman"):
    """Generates a random experiment name with prefix and padded ID."""
    run_id = f"{random.randint(1, 999):03d}"
    timestamp = datetime.now().strftime("%y%m%d") # YYMMDD
    return f"{prefix}_{timestamp}_{run_id}"

def _normalize_model_name(model_name: str) -> str:
    """
    Normalizes model name to be used as a safe filename.
    Example: 'ultralytics/yolo11n-cls.pt' -> 'yolo11n_cls'
    """
    # 1. Use Path to extract the filename without the extension
    # Path('path/to/model.pt').stem -> 'model'
    clean_name = Path(model_name).stem
    
    # 2. Replace any non-alphanumeric character with an underscore
    # This handles dots, slashes, dashes, and even weird characters
    clean_name = re.sub(r'[^a-zA-Z0-9]', '_', clean_name)
    
    # 3. Clean up: Collapse multiple underscores into one, 
    # trim edges, and lowercase for consistency
    clean_name = re.sub(r'_+', '_', clean_name).strip('_').lower()
    
    return clean_name

In [4]:
from ultralytics import YOLO

# Load a YOLO classification model (n is the fastest/smallest)
model_name = "yolo11s-cls.pt"
model = YOLO(model_name)  # or "yolov8n-cls.pt"

# Generates a 3-digit padded number (001, 042, etc.)
_exp_name = _normalize_model_name(model_name=model_name)
exp_name = random_exp_name(prefix=_exp_name)

wandb.init(
    project=WANDB_PROJECT, 
    name=exp_name, 
    job_type=WANDB_JOB_TYPE,
    dir=str(LOG_DIR) 
)

results = model.train(
    data=str(DATA_DIR / "yolo_dataset"),
    epochs=100,
    imgsz=224,             
    batch=64,               # Increased for stability
    workers=8,
    device=0,
    amp=True,
    
    # --- Optimization (Lower LR for AdamW) ---
    optimizer="AdamW",     
    lr0=0.001,              # Lowered from 0.01 to fix the "V-dip"
    lrf=0.01,
    warmup_epochs=5.0,      # Longer transition period
    patience=15,            # Give it more time to recover from plateaus
    
    # --- Data Augmentation (Face Optimized) ---
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10.0,
    translate=0.1,
    scale=0.2,              # Reduced from 0.5 to keep faces larger
    shear=1.0,              # Reduced to prevent extreme face distortion
    fliplr=0.5,
    flipud=0.0,
    auto_augment="randaugment", 
    erasing=0.2,
    
    # --- Saving & Checkpoints ---
    project=LOG_DIR / WANDB_PROJECT,
    name=exp_name,
    save=True,
    save_period=5,
    val=True,
    plots=True,
    verbose=True,

    # --- Regularization ---
    dropout=0.15,           # Slightly increased
    label_smoothing=0.1,    # Added to improve generalization
    pretrained=True
)


# 5. Finalize
metrics = model.val()


wandb.finish()

New https://pypi.org/project/ultralytics/8.3.247 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.10.18 torch-2.4.1+cu121 CUDA:0 (Quadro P2000, 4034MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentu

lr/pg0,█▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁
lr/pg2,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▁
metrics/accuracy_top1,▂▁▂▅▆▅▆▇▇▇▇▇▇██▇▇███████████████████████
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,▄█▆▆▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,█▇▆▇█▄▄▃▃▃▃▄▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg0,0.0006


Ultralytics 8.3.228 🚀 Python-3.10.18 torch-2.4.1+cu121 CUDA:0 (Quadro P2000, 4034MiB)
YOLO11s-cls summary (fused): 47 layers, 5,436,690 parameters, 0 gradients, 12.0 GFLOPs
train: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/train... found 5973 images in 2 classes ✅ 
val: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/val... found 1706 images in 2 classes ✅ 
test: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/test... found 855 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 411.2±130.4 MB/s, size: 9.2 KB)
val: Scanning /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/val... 1693 images, 13 corrupt: 100% ━━━━━━━━━━━━ 1693/1693 1.9Mit/s 0.0s0s
val: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/

In [5]:
# 1. Validate on the standard validation set (happens automatically, but good to be explicit)
print("--- Validation Set Results ---")
val_metrics = model.val(split='val') 

# 2. Validate on the TEST set (Your final unbiased metrics)
print("--- Test Set Results ---")
test_metrics = model.val(split='test')

# 3. Print the top-1 accuracy for the test set
print(f"Final Test Top-1 Accuracy: {test_metrics.top1:.4f}")

--- Validation Set Results ---
Ultralytics 8.3.228 🚀 Python-3.10.18 torch-2.4.1+cu121 CUDA:0 (Quadro P2000, 4034MiB)
train: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/train... found 5973 images in 2 classes ✅ 
val: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/val... found 1706 images in 2 classes ✅ 
test: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/test... found 855 images in 2 classes ✅ 
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 258.4±61.4 MB/s, size: 9.2 KB)
val: Scanning /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/val... 1693 images, 13 corrupt: 100% ━━━━━━━━━━━━ 1693/1693 1.8Mit/s 0.0s0s
val: /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/val/non_human/image_000823.png: ignoring cor

In [6]:
onnx_path = model.export(format="onnx", imgsz=224) 
print(f"Model exported to: {onnx_path}")

Ultralytics 8.3.228 🚀 Python-3.10.18 torch-2.4.1+cu121 CPU (Intel Core(TM) i9-8950HK 2.90GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/home/octoopt/workspace/projects/learn-from-basics/compvision_learning/logs/human-non-human-classification/yolo11s_cls_260104_086/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 2) (10.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71'] not found, attempting AutoUpdate...
Using Python 3.10.18 environment at: /home/octoopt/anaconda3/envs/turlio
Resolved 10 packages in 459ms
Prepared 4 packages in 1.27s
Uninstalled 2 packages in 1.89s
Installed 4 packages in 197ms
 + colorama==0.4.6
 - onnx==1.20.0
 + onnx==1.19.1
 + onnxslim==0.1.82
 - sympy==1.12
 + sympy==1.14.0

requirements: AutoUpdate success ✅ 4.2s
WARNING ⚠️ requirements: Restart runtime or rerun command

In [7]:
import os
import cv2
import numpy as np
import onnxruntime as ort
from pathlib import Path
from tqdm import tqdm

def preprocess_image(img_path, target_size=(224, 224)):
    """Preprocesses image to match YOLOv11-cls requirements."""
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    
    # Normalize: Convert to float32 and scale to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    # Channel First (H, W, C) -> (C, H, W)
    img = img.transpose(2, 0, 1)
    
    # Add Batch dimension (C, H, W) -> (1, C, H, W)
    img = np.expand_dims(img, axis=0)
    return img

def evaluate_onnx_classification(onnx_model_path, test_dir):
    # 1. Start ONNX Runtime Session
    providers = ['CPUExecutionProvider'] # or ['CUDAExecutionProvider'] if you have GPU ORT
    session = ort.InferenceSession(onnx_model_path, providers=providers)
    input_name = session.get_inputs()[0].name
    
    test_path = Path(test_dir)
    # Get class names from folders
    classes = sorted([d.name for d in test_path.iterdir() if d.is_dir()])
    class_to_idx = {cls: i for i, cls in enumerate(classes)}
    
    correct = 0
    total = 0
    
    print(f"Evaluating {onnx_model_path} on {test_dir}...")
    
    # 2. Iterate through class folders
    for class_name in classes:
        class_dir = test_path / class_name
        image_paths = list(class_dir.glob("*.[jJ][pP][gG]")) + list(class_dir.glob("*.[pP][nN][gG]"))
        
        for img_path in tqdm(image_paths, desc=f"Class: {class_name}"):
            # Preprocess
            input_tensor = preprocess_image(img_path)
            
            # Inference
            outputs = session.run(None, {input_name: input_tensor})
            
            # Post-process (Softmax/Argmax)
            logits = outputs[0]
            predicted_idx = np.argmax(logits)
            
            # Check accuracy
            if predicted_idx == class_to_idx[class_name]:
                correct += 1
            total += 1

    accuracy = correct / total if total > 0 else 0
    print("-" * 30)
    print(f"Test Accuracy: {accuracy:.4f} ({correct}/{total})")
    print("-" * 30)
    return accuracy

In [9]:
onnx_path

'/home/octoopt/workspace/projects/learn-from-basics/compvision_learning/logs/human-non-human-classification/yolo11s_cls_260104_086/weights/best.onnx'

In [10]:
# --- Execute ---
ONNX_PATH = onnx_path
TEST_DATA_DIR = DATA_DIR / "yolo_dataset/test"

evaluate_onnx_classification(ONNX_PATH, TEST_DATA_DIR)

Evaluating /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/logs/human-non-human-classification/yolo11s_cls_260104_086/weights/best.onnx on /home/octoopt/workspace/projects/learn-from-basics/compvision_learning/data/human_nonhuman/yolo_dataset/test...


Class: non_human: 100%|██████████| 418/418 [00:03<00:00, 115.24it/s]

------------------------------
Test Accuracy: 0.9824 (836/851)
------------------------------


0.982373678025852

In [12]:
from huggingface_hub import HfApi, login
import os

# Login using token from .env or manual input
hf_login(token=os.getenv("HF_TOKEN")) # You already have this import as hf_login

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [17]:
import os
from huggingface_hub import HfApi

# 1. Configuration (Dynamically linked to your training variables)
repo_id = "8Opt/yolo11-human-nonhuman-cls"

# Construct paths using the log_dir and exp_name from your training cell
# Adjust the 'yolo_classification' part if you changed your project name in model.train()
weights_dir = LOG_DIR / "human-non-human-classification" / exp_name / "weights"

model_pt_path = str(weights_dir / "best.pt")
model_onnx_path = str(weights_dir / "best.onnx")

In [15]:
api = HfApi()

# 2. Create/Update Repo
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

# 3. Upload Model Files
print(f"Uploading models from {exp_name}...")

api.upload_file(
    path_or_fileobj=model_pt_path,
    path_in_repo="best.pt",
    repo_id=repo_id,
)

if os.path.exists(model_onnx_path):
    api.upload_file(
        path_or_fileobj=model_onnx_path,
        path_in_repo="model.onnx",
        repo_id=repo_id,
    )

# 4. Create and Upload a professional Model Card (README.md)
model_card_content = f"""
---
license: mit
library_name: ultralytics
tags:
- real-time
- yolo
- yolo11
- classification
- human-detection
metrics:
- accuracy: 0.9824
---

# YOLOv11n Human vs Non-Human Classification

This is a fine-tuned **YOLOv11s-cls** model for binary classification.

## 📊 Performance
- **Test Accuracy**: 98.24% (836/851 correctly classified)
- **Dataset**: Human faces vs. Non-human (Statues, Art, Anime, Gaming)
- **Experiment ID**: `{exp_name}`

## 🚀 Model Usage

### 1. Using Ultralytics (Python)
The easiest way to use the trained PyTorch model:

```python
from ultralytics import YOLO

# Load the model
model = YOLO('path/to/best.pt')

# Predict on an image
results = model('image.jpg')

# Process results
for result in results:
    probs = result.probs  # Probs object for classification outputs
    print(f"Top-1 class: {{result.names[probs.top1]}}")
    print(f"Confidence: {{probs.top1conf:.2f}}")
```

### 2. Using YOLO CLI
```bash
yolo classify predict model=path/to/best.pt source='image.jpg'
```

### 3. Using ONNX Runtime
For production or edge deployment without PyTorch:

```python
import onnxruntime as ort
import numpy as np
import cv2

# Initialize session
session = ort.InferenceSession("model.onnx")

# Preprocess (Resize to 224x224, normalize [0, 1], CHW format)
img = cv2.imread("image.jpg")
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (224, 224))
img = img.astype(np.float32) / 255.0
img = img.transpose(2, 0, 1)[np.newaxis, :]  # Add batch dim

# Run Inference
outputs = session.run(None, {{"images": img}})
predicted_idx = np.argmax(outputs[0]) 
print(f"Predicted Class Index: {{predicted_idx}}")
```

## 📈 Baseline Metrics (04/01/2026)
- **Model**: YOLOv11n-cls
- **Input Size**: 224x224
- **Test Accuracy**: 98.24%
"""

with open("temp_README.md", "w") as f: f.write(model_card_content)

api.upload_file( path_or_fileobj="temp_README.md", path_in_repo="README.md", repo_id=repo_id, )

print(f"✅ Model successfully pushed to: https://huggingface.co/{repo_id}")

Uploading models from yolo11s_cls_260104_086...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ Model successfully pushed to: https://huggingface.co/8Opt/yolo11n-human-nonhuman-cls
